In [ ]:
import pandas as pd
import re
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import accelerate
import torch.nn as nn
from torch.optim import Adam

In [1]:
import pandas as pd
import re

In [ ]:
from tqdm.auto import tqdm

In [2]:
train_data = pd.read_csv('hpc_space/MBERT5/train_data_mid_5.csv')
#val_data = pd.read_csv('hpc_space/MBERT1/val_data_mid_1.csv')

In [3]:
# train_data = train_data.sample(frac=1, random_state=99)

In [4]:
print(train_data)

                                                   text  outcome  Geslacht  \
0     Orienterend lab + UDS aangevraagd\n\n\n\nNoodb...        0         1   
1     PO/ We zien een matig verzorgde, obese vrouw m...        0         0   
2     \n\n\n\nTelefonisch uitleg gegeven over de ECT...        0         1   
3     \n\n\n\nVinkers, dienstdoende psychiatrie.\n\n...        0         0   
4     1. Psychische problemen \nRvO: Angst met psych...        0         1   
...                                                 ...      ...       ...   
2795  1. Psychische problemen:\nWeinig in beeld, af ...        0         1   
2796  \n\n\n\n\n\n\n\nMDO beleid:\n- opname vandaag\...        0         1   
2797  Dhr kwam rond 14.30 op de afdeling samen met <...        0         1   
2798  <PERSOON-1>\n\nMevr kwam samen met haar man en...        1         0   
2799  \n\n\n\nOpnamegesprek in dienst\nAanwezig: pte...        1         0   

     Combined  
0         0_1  
1         0_0  
2         0_1  

In [5]:
def clean_txt(text):
    text = re.sub("'", "", text)
    text = re.sub("(\\W)+", " ", text)
    return text

In [ ]:
train_data['text'] = train_data.text.apply(clean_txt)
#val_data['text'] = val_data.text.apply(clean_txt)

In [ ]:
def replace_gendered_words(text, gender):
    gendered_word_mapping = {
        0: {"zij": "hij", "Zij": "Hij", "ze": "hij", "Ze": "Hij", "haar": "hem", "Haar": "Hem", "mw": "dhr", "Mw": "Dhr", "vrouw": "man", "Vrouw": "Man", "patiente": "patient", "Patiente": "Patient", "mevrouw": "meneer", "Mevrouw": "Meneer", "mevr": "mr", "Mevr": "Mr", "meisje": "jongen", "Meisje": "Jongen", "dame": "heer", "Dame": "Heer"},
        1: {"hij": "zij", "Hij": "Zij", "hem": "haar", "Hem": "Haar", "zijn": "haar", "Zijn":"Haar", "dhr": "mw", "Dhr": "Mw", "man": "vrouw", "Man": "Vrouw", "patient": "patiente", "Patient": "Patiente", "meneer": "mevrouw", "Meneer": "Mevrouw", "mr": "mevr", "Mr": "Mevr", "jongen": "meisje", "Jongen": "Meisje", "heer": "dame", "Heer": "Dame", "hr":"mw", "Hr": "Mw"},
    }

    words = text.split()

    for i in range(len(words)):
        word = words[i]
        if word in gendered_word_mapping[gender]:
            words[i] = gendered_word_mapping[gender][word]

    return ' '.join(words)

if __name__ == "__main__":

    # Replace gendered words in the 'text' column based on the 'Geslacht' column
    train_data['text'] = train_data.apply(lambda row: replace_gendered_words(row['text'], row['Geslacht']), axis=1)

    # Print the modified DataFrame
    print("Modified DataFrame:")
    print(train_data)

In [ ]:
print(train_data)

In [ ]:
# if __name__ == "__main__":

#     # Replace gendered words in the 'text' column based on the 'Geslacht' column
#     val_data['text'] = val_data.apply(lambda row: replace_gendered_words(row['text'], row['Geslacht']), axis=1)

#     # Print the modified DataFrame
#     print("Modified DataFrame:")
#     print(val_data)

In [ ]:
train_data_orig = pd.read_csv('hpc_space/MBERT4/train_data_mid_4.csv')
#val_data_orig = pd.read_csv('hpc_space/MBERT5/val_data_mid_5.csv')

In [ ]:
train_data_orig['text'] = train_data_orig.text.apply(clean_txt)
# val_data_orig['text'] = val_data_orig.text.apply(clean_txt)

In [ ]:
print(train_data)

In [ ]:
print(train_data_orig)

In [ ]:
train_data = pd.concat([train_data_orig, train_data])

In [ ]:
train_data.to_csv('train_data_augmented_4.csv', index=False)

In [ ]:
val_data = pd.concat([val_data_orig, val_data])

In [ ]:
print(train_data)

In [ ]:
def get_split(text1):
    l_total = []
    chunk_size = 500
    overlap = 50

    words = text1.split()

    for start_idx in range(0, len(words), chunk_size - overlap):
        end_idx = start_idx + chunk_size
        l_parcial = words[start_idx:end_idx]
        l_total.append(" ".join(l_parcial))

    return l_total

In [ ]:
train_data['text_split'] = train_data['text'].apply(get_split)
val_data['text_split'] = val_data['text'].apply(get_split)

In [ ]:
train_l = []
label_l = []
index_l = []

for idx, row in train_data.iterrows():
    for l in row['text_split']:
        train_l.append(l)
        label_l.append(row['outcome'])
        index_l.append(idx)

In [ ]:
val_l = []
val_label_l = []
val_index_l = []

for idx, row in val_data.iterrows():
    for l in row['text_split']:
        val_l.append(l)
        val_label_l.append(row['outcome'])
        val_index_l.append(idx)

In [ ]:
train_df = pd.DataFrame({'text':train_l, 'label':label_l})
train_df.head()

In [ ]:
val_df = pd.DataFrame({'text':val_l, 'label':val_label_l})
val_df.head()

In [ ]:
train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()

In [ ]:
eval_texts = val_df['text'].tolist()
eval_labels = val_df['label'].tolist()

In [ ]:
import torchvision
import torchaudio

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("JoppeK/MBERT_copy")
model = AutoModelForSequenceClassification.from_pretrained("JoppeK/MBERT_copy", num_labels=2)

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

In [ ]:
from datasets import Dataset

In [ ]:
raw_dataset_train = Dataset.from_pandas(train_df.reset_index(drop=True))
print(raw_dataset_train)

In [ ]:
raw_dataset_val = Dataset.from_pandas(val_df.reset_index(drop=True))
print(raw_dataset_val)

In [ ]:
tokenized_dataset_train = raw_dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_train

In [ ]:
tokenized_dataset_val = raw_dataset_val.map(tokenize_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_dataset_train = tokenized_dataset_train.remove_columns(["text"])
tokenized_dataset_train = tokenized_dataset_train.rename_column("label", "labels")
tokenized_dataset_train.set_format("torch")
tokenized_dataset_train.column_names

In [ ]:
tokenized_dataset_val = tokenized_dataset_val.remove_columns(["text"])
tokenized_dataset_val = tokenized_dataset_val.rename_column("label", "labels")
tokenized_dataset_val.set_format("torch")
tokenized_dataset_val.column_names

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset_train, shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset_val, batch_size=16, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-6)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
from tqdm.auto import tqdm
from transformers import get_scheduler
from accelerate import Accelerator
accelerator = Accelerator()

train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        class_weights = torch.tensor([1.0, 2.0], dtype=torch.float32).to(accelerator.device)
        weighted_loss = loss * class_weights[batch["labels"]]
        loss = weighted_loss.mean()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("MBERT_augmented_5")

In [ ]:
tokenizer.push_to_hub("MBERT_augmented_5")